<h1 style="text-align:center;"> Python y Cosmología </h1> 

<h1 style="text-align:center;">
Análisis de datos en Cosmología usando Jupyter Notebook </h1> 


<p style="font-size:20px">
<ol>
<li> Dar una introducción al análisis de datos en Cosmología del CMB
<li> Acceder al archivo de Planck http://pla.esac.esa.int/pla/#home.
<li> Ejemplos.
<li> Paquetes adicionales: AstroML, scipy, healpy, camb, astropy.
<ol>
 
 http://www.astroml.org/
 
 https://camb.readthedocs.io/en/latest/camb.html
 
 https://healpy.readthedocs.io/en/latest/tutorial.html
 
 http://www.astropy.org/

Datos:
Utilizaremos 5 archivos de datos para los ejemplos:

3 mapas de CMB -> SMICA, Commander y NILC Full Mission maps download http://pla.esac.esa.int/pla/#maps

COM_CMB_IQU-smica_1024_R2.02_full.fits

COM_CMB_IQU-commander_1024_R2.02_full.fits

COM_CMB_IQU-nilc_1024_R2.02_full.fits

1 mapa de Foregrounds -> Dust -> Commander maps download http://pla.esac.esa.int/pla/#maps

COM_CompMap_dust-commander_0256_R2.00.fits

1 CMB Angular Power Spectrum http://pla.esac.esa.int/pla/#cosmology

COM_PowerSpect_CMB-base-plikHM-TT-lowTEB-minimum-theory_R2.02.txt

In [ ]:
#Set/path/to/data/dir
from os.path import expanduser
home = expanduser("~")
data_dir = '%s/Astrofisica Computacional/cosmology/' % home

In [ ]:
%matplotlib inline
import sys, platform, os
from matplotlib import pyplot as plt
import numpy as np
import camb
import math
import healpy as hp 
from tqdm import tqdm

#For spherical harmonics
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import sph_harm

from scipy.fftpack import ifftn, fftn
import matplotlib.cm as cm
from math import pi, sin, cos, sqrt, log, floor

plt.rcParams['figure.figsize'] = (12.0, 12.0)

<h1 style="text-align:center;"> The Fourier Transform </h1>

La transformada de Fourier es una técnica importante para el tratamiento de señales, en Cosmología y en otros campos.

En Cosmología se utiliza para calculare el espectro de potencia de señales (la distribución de la potencia en los componentes de frecuencia presentes en la señal.

Facilita la resolución de problemas muy complicados utilizadon "trucos".

Cualquier función compleja continua ($-\infty, +\infty$) se puede representar por una sumatoria de ondas plans clan $\mathrm{e}^{iwt}$. 

(Formula de Euler: $e^{iwt} = \cos(wt) + i\sin(wt)$)

Claro que en la práctica tenemos un intervalo finito y un conjunto discreto de puntos para evaluar.


In [ ]:
plt.rcParams.update({'font.size': 22})

x = np.linspace(0, 4*pi, 100)
y = np.sin(x)
y2 = np.sin(2*x)

subx = np.linspace(1*pi, 3*pi, 20)
suby = np.sin(subx)
suby2 = np.sin(2*subx)

fig, ax = plt.subplots(1,2, figsize = (20,10))

ax[0].plot(x, y, lw=0.2)
ax[0].plot(subx, suby, 'ro')

ax[1].plot(x, y2, lw=0.2)
ax[1].plot(subx, suby2, 'ro')

#A bunch of lines and text

for i in range(0,2):
    ax[i].plot([pi, pi], [-0.5, sin(11*pi/10)], 'k-', lw=1)
    ax[i].plot([11*pi/10., 11*pi/10.], [sin(11*pi/10), -0.5], 'k-', lw=1)
    ax[i].plot([pi, 3*pi], [0,0], 'k-', lw=1)
    ax[i].plot([pi, 11*pi/10.], [-0.5,-0.5], 'k-', lw=1)
    ax[i].text(9*pi/10., -0.6, '$\Delta t$')
    ax[i].text(6, 0.05, '$L$')
    ax[i].text(3*pi, 0.05, '$y_{N-1}$')
    ax[i].text(pi, 0.05, '$y_0$')
 
    ax[i].set_ylabel('$y$')



La Tranformada de Fourier Discreta (DFT) es: $Y_k = \sum\limits_{n=0}^{N - 1}y_n\mathrm{e}^{-2\pi ikn/N}$

Transforma un vector $y_n$ de N números complejos en un vector $Y_k$ de N números complejos.

La inversa de la DFT es: $y_n = \frac{1}{N}\sum\limits_{m=0}^{N - 1}Y_k\mathrm{e}^{2\pi ikn/N}$

La Transformada Rápida de Fourier (Fast Fourier Transform FFT) es un algoritmo que se aprovecha de las multiplicaciones repedias para acelear el cálculo de la DFT 

Cooley & Tukey 1965 http://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/

DFT $O(N^2)$ 

FFT $O(N\log N)$

In [ ]:
plt.rcParams.update({'font.size': 14})

# Code copied from:
# https://docs.scipy.org/doc/scipy/reference/tutorial/fftpack.html

N = 100
f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, sharex='col', sharey='row')
xf = np.zeros((N,N))
xf[0, 5] = 1
# xf[0, N-5] = 1
Z = ifftn(xf)
ax1.imshow(xf, cmap=cm.Reds)
ax4.imshow(np.real(Z), cmap=cm.gray)


xf = np.zeros((N, N))
xf[5, 0] = 1
# xf[N-5, 0] = 1
Z = ifftn(xf)
ax2.imshow(xf, cmap=cm.Reds)
ax5.imshow(np.real(Z), cmap=cm.gray)


xf = np.zeros((N, N))
xf[5, 10] = 1
xf[N-5, N-10] = 1
Z = ifftn(xf)
ax3.imshow(xf, cmap=cm.Reds)
ax6.imshow(np.real(Z), cmap=cm.gray)


plt.show()

<h1 style="text-align:center;"> The CMB </h1>

Qué es el CMB?

La teoría del Big Bang predice un campo de radiación a una temperatura de 2.72 K liberada alrededor de 380,000 años después del Big Bang cuando la temperatura del Universo bajo lo suficiente para que la materia y la energía se separaran. 

Es un campo uniforme e isotrópico, con un espectro de cuerpo negro. En este campo de radiación existen fluctuaciones de temperatura originadas por la distribución de materia en el Universo $\mathcal{O}(10^{-5}K)$

También existen fluctuaciones en la polarización de unos cuantos órdenes de magnitud inferior.

Fue el gran reto de la Cosmología Observacional de finales del Siglo XX.





In [ ]:
#Before we start plotting the CMB let's set a nice cmap
cmap = cm.RdBu_r
cmap.set_under("w") # sets background to white

In [ ]:
planck_IQU_SMICA = hp.read_map('COM_CMB_IQU-smica_1024_R2.02_full.fits')
fig = plt.figure(1)
hp.mollview(planck_IQU_SMICA, min=-0.0003, max=0.0003, title='Planck Temperature Map',
            fig=1, unit='K',cmap=cmap)

<h1 style="text-align:center;"> The Data </h1>



El archivo de Plank es público: http://pla.esac.esa.int/pla/#home

O el de WMAP (con menos resolución): https://lambda.gsfc.nasa.gov/product/map/current/


<h1 style="text-align:center;"> Gaussian Random Fields  </h1>

Como describimos el CMB?

Un campo aleatorio continuo 2D $y(\hat{q}) = \delta T(\hat{q})/T_{CMB}$ donde $\hat{q}$ cubre la esfera celeste.

Esto se mapea a un vector ${\bf y} = [y_1,...,y_n]$ donde $N$ es el número de pixels.

El CMB es un Campo Aleatorio Gaussiano (Gaussian Random Field),  o al menos muy cerca de serlo, así que el vector ${\bf y}$ es una multivariable aleatoria con media gaussiana cero y una Probability Distribution Function:

$P(y_1,...,y_n) =\frac{1}{(2\pi)^{N/2} | C|^{1/2}}\exp(-\frac{1}{2}{\bf y.C^{-1}.y^{\dagger}})$

donde $C$ es la matriz de covarianza $C = \langle {\bf y.y}^{\dagger}\rangle$

In [ ]:
from scipy.stats import norm
import matplotlib.mlab as mlab

# best fit of data
(mu, sigma) = norm.fit(planck_IQU_SMICA)

# the histogram of the data
n, bins, patches = plt.hist(planck_IQU_SMICA, 60, normed=1, facecolor='r', alpha=0.25)

# add a 'best fit' line
y = mlab.normpdf( bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

plt.xlabel('Temperature/K')
plt.ylabel('Frequency')
plt.title(r'Histogram of $12N_{side}^2$ pixels from the Planck SMICA map', y=1.08)
plt.xlim(-0.0005, 0.0005)

plt.legend()

plt.show()

<h1 style="text-align:center;"> Predicting the CMB power spectra </h1>

Dadas las condiciones iniciales y los valores de los parámetros cosmológicos, códigos numéricos de CMB Boltzmann como CAMB hacen evolucionar las perturbaciones iniciales utilizando las ecuaciones de Boltzmann para predecir el espectro del CMB. 

Para mayor información, vean la documentación de CAMB
https://camb.readthedocs.io/en/latest/
<img src="camb.png">
Source: camb.info

In [ ]:
#Object storing the parameters for a CAMB calculation, including cosmological parameters
cp = camb.model.CAMBparams() 

#Sets cosmological parameters in terms of physical densities and parameters used in Planck 2015 analysis. 
T0 = 2.7255
cp.set_cosmology(TCMB=T0)

#Set parameters to get CMB power spectra accurate to specific a l_max.
cp.set_for_lmax(2500)

#Set the InitialPower primordial power spectrum parameters
cp.InitPower.set_params()

In [ ]:
#Parameters used by Fortran CAMB
print(cp)

In [ ]:
cp.validate()

Para probar sus parámetros cosmológicos, pueden visitar: https://lambda.gsfc.nasa.gov/toolbox/tb_camb_form.cfm

In [ ]:
#Calculate results for specified parameters and return CAMBdata instance for getting results.
results = camb.get_results(cp)

In [ ]:
#Get dictionary of CMB power spectra
powers = results.get_cmb_power_spectra(cp)
for name in powers: print(name)

<h1 style="text-align:center;"> Experiments </h1>

Read more: https://wiki.cosmos.esa.int/planckpla2015/index.php/Main_Page

Data: http://pla.esac.esa.int/pla/#cosmology

In [ ]:
#read in theoretical best fit power spectra from Planck 2015 data
planck_theory_cl = np.loadtxt("COM_PowerSpect_CMB-base-plikHM-TT-lowTEB-minimum-theory_R2.02.txt")

In [ ]:
#Plot the total lensed CMB power spectra against the theoretical best fit
plt.rcParams.update({'font.size': 14})

#All power spectra are l(l+1)C_l/2pi numpy arrays (0..lmax, 0..3), where 0..3 index are TT, EE, BB, TE.
totCL = powers['total']
print(totCL.shape)

T0sq = T0**2
totCL = T0sq*totCL

#Convert from K^2 to microK^2
totCL = totCL*1e12

ls = np.arange(totCL.shape[0])

fig, ax = plt.subplots(3,1, figsize = (12,20))
ax[0].set_xlabel('$l$')
ax[0].set_ylabel('$l(l+1)C_l/2\pi$')
ax[0].plot(planck_theory_cl[:,0], planck_theory_cl[:,1], '-r', label = ' Planck best fit')
ax[0].plot(ls,totCL[:,0], color='k', label = 'CAMB test')
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].legend(loc='lower left')
ax[0].set_title('$TT$')

ax[1].plot(ls,totCL[:,1], color='k')
ax[1].set_xlabel('$l$')
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_title(r'$EE$')

ax[2].plot(ls,totCL[:,3], color='k')
ax[2].set_xlabel('$l$')
ax[2].set_title(r'$TE$')
ax[2].set_xscale('log')

for ax in ax.reshape(-1): ax.set_xlim([2,2500])


<h1 style="text-align:center;"> Spherical Harmonics </h1>

Los armónicos esféricos son funciones propias del cuadrado del momento angular orbital; los primeros son: 

$Y_{00}(\theta,\phi) = \frac{1}{\sqrt{4\pi}}$

$Y_{10}(\theta,\phi) = \mathrm{i}\sqrt{\frac{3}{4\pi}}\cos\theta$

$Y_{1,\pm1}(\theta,\phi) = \mp \mathrm{i}\sqrt{\frac{3}{8\pi}}\sin\theta \,\mathrm{e}^{\pm i\phi}$

$Y_{20}(\theta,\phi) = \sqrt{\frac{5}{16\pi}}(1 - 3\cos^2\theta)$

$Y_{2,\pm1}(\theta,\phi) = \pm \mathrm{i}\sqrt{\frac{15}{8\pi}}\cos\theta\sin\theta \,\mathrm{e}^{\pm i\phi}$

$Y_{2,\pm2}(\theta,\phi) = -\sqrt{\frac{15}{32\pi}}\sin^2\theta \,\mathrm{e}^{\pm 2i\phi}$

La Temperatura se define sobre una esfera, es decir, es una función de $\theta$ y $\phi$ así que se puede descomponer en la suma de armónicos esféricos:

$T(\hat{n}) = \sum\limits_{l,m}a_{T,lm}Y_{lm}(\hat{n})$

Esto es análogo a la descomposición de Fourier en el plano, es decir, campos 2D o 3D son función de coordenadas espaciales:

La amplitud $a_{lm}$ son coeficienes complejos de los armónicos esféricos. Tienen una distribución Gaussiana en el modelo estándar de inflación. 

$l$ se relaciona a la escala angular en el cielo.

$m$ se relaciona con la orientación de este modo en el cielo

La función de densidad de probabilidad (PDF) de la distribución $a_{lm}$ es

$P(a_{lm}) = \frac{1}{\sqrt{2\pi C_l}}e^{-a^2_{lm}/2C_l} $

$a_{lm}$ son variables aleatorias con media Gaussiana cero

$C_l$ es el espectro de potencia (la varianza de la distribución.)

La varianza del campo en una dada escala angular es:  

$\langle a^*_{l'm'}a_{lm} \rangle = C_l\delta_{l'l}\delta_{m'm}$

Solamente tenemos un universo así que nuestra mejor estimación viene del promedio sobre $m$

Isotropía Estadística: No existe correlación entre las $a_{lm}$ a menos que $l=l'$ y $m=m'$

<h1 style="text-align:center;"> Spherical Harmonics </h1>

In [ ]:
# http://scipython.com/book/chapter-8-scipy/examples/visualizing-the-spherical-harmonics/
# Example code used from the book Learning Scientific Programming with Python 
# published by Cambridge University Press (ISBN: 9781107428225).

phi = np.linspace(0, np.pi, 100) #Polar (colatitudinal) coordinate
theta = np.linspace(0, 2*np.pi, 100) #Azimuthal (longitudinal) coordinate
phi, theta = np.meshgrid(phi, theta)

In [ ]:
# The Cartesian coordinates of the unit sphere
# https://en.wikipedia.org/wiki/Spherical_coordinate_system
x = np.sin(phi) * np.cos(theta)
y = np.sin(phi) * np.sin(theta)
z = np.cos(phi)

In [ ]:
def norm_harmonic(l, m, theta, phi):
    # Calculate the spherical harmonic Y(l,m) and normalize to [0,1]
    fcolors = sph_harm(m, l, theta, phi).real
    fmax, fmin = fcolors.max(), fcolors.min()
    fcolors = (fcolors - fmin)/(fmax - fmin)
    return fcolors

In [ ]:
def plot_harmonic(fcolors, x, y, z):
    # Set the aspect ratio to 1 so our sphere looks spherical
    fig = plt.figure(figsize=plt.figaspect(1.))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=cm.RdBu_r(fcolors))
    # Turn off the axis planes
    ax.set_axis_off()
    plt.show()

In [ ]:
def plot_ax(fig, fcolors, x, y, z, numplots, pos):
    ax = fig.add_subplot(1, numplots, pos, projection='3d', aspect=1.0)
    ax.plot_surface(x, y, z,  rstride=1, cstride=1, facecolors=cm.RdBu_r(fcolors))
    # Turn off the axis planes
    ax.set_axis_off()

In [ ]:
def plot_l(l):
    fig = plt.figure(figsize=(12, 12))
    for m in range(0, l + 1):
        plot_ax(fig, norm_harmonic(l, m, theta, phi), x, y, z, l + 1, m + 1)
    plt.show()

Ahora hagamos un plot:

In [ ]:
l = 0; m = 0
fcolors = sph_harm(m, l, theta, phi).real
plot_harmonic(fcolors/fcolors, x, y, z)

In [ ]:
plot_l(1)

In [ ]:
plot_l(2)

In [ ]:
plot_l(3)

In [ ]:
plot_l(4)

Como se relacionan los armónicos esféricos con el CMB?


http://spud.spa.umn.edu/~pryke/logbook/20000922/

http://spud.spa.umn.edu/~pryke/logbook/20000922/individual_cmb.html

http://spud.spa.umn.edu/~pryke/logbook/20000922/cumulative_cmb.html

Cuál es la relación entre $l$ y $\theta$? $l \approx 180/\theta$





In [ ]:
plt.plot(planck_theory_cl[:,0], planck_theory_cl[:,1], '-r')
plt.xlim(0,2500)
plt.show()

<h1 style="text-align:center;"> Instrumental Effects </h1>

Gaussian beam in harmonic space
$B_l(\theta_s) = exp(-2(l+\frac{1}{2})^2sin^2(\frac{\theta_s}{2}))$ 

where $\theta_s = \theta_{FWHM}/\sqrt{8\ln 2}$

The power spectrum of the observed map $C_\ell(map)$ is the convolution of the beam window function $B_l$ with the power spectrum of the sky $C_l(sky)$

$C_l(map)$ = $C_l(sky)*B_l(\theta_s)$ 

The Planck SMICA map has an effective beam window function described here:

https://wiki.cosmos.esa.int/planckpla/index.php/CMB_and_astrophysical_component_maps
https://wiki.cosmos.esa.int/planckpla2015/index.php/CMB_and_astrophysical_component_maps#Polarization_products

In [ ]:
#Calculating the beam window function
ls = np.arange(1024)
beam_arcmin = 30.0

def B_l(beam_arcmin, ls):
    theta_fwhm = ((beam_arcmin/60.0)/180.0)*math.pi #angle in radians 
    theta_s = theta_fwhm/(math.sqrt(8*math.log(2)))
    return np.exp(-2*(ls + 0.5)**2*(math.sin(theta_s/2.0))**2)

#Let's plot it
fig, ax = plt.subplots(1,1, figsize = (10,10))
ax.plot(ls, B_l(beam_arcmin, ls), color='k')
ax.set_xlabel(r'$\ell$')
ax.set_ylabel(r'$B_\ell(\theta_s)$')
ax.set_title(r"Beam Window Function for $\theta_{FWHM}$ of 30'")

<h1 style="text-align:center;"> White Noise </h1>

In [ ]:
white_noise = np.ma.asarray(np.random.normal(0, 0.0001, 12*1024**2))

In [ ]:
fig = plt.figure(1)
hp.mollview(white_noise, min=-0.0003, max=0.0003, title='White Noise Map',
            fig=1, unit=r'Temperature/K', cmap=cmap)

In [ ]:
plt.hist(white_noise,  bins=np.arange(-0.0005, 0.0005, 0.00002), color='b', alpha = 0.2 ) 
plt.xlim(-0.0005, 0.0005)
plt.xlabel('Temperature/K')
plt.ylabel('Frequency')
plt.title(r'Histogram of $12N_{side}^2$ random samples from a normal (Gaussian) distribution', y=1.08)
plt.show()

In [ ]:
plt.plot(planck_theory_cl[:,0], planck_theory_cl[:,1], '-r')
plt.xlim(0,2500)
plt.show()

In [ ]:
#http://pla.esac.esa.int/pla/#maps
planck_IQU_SMICA = hp.read_map('%sCOM_CMB_IQU-smica_1024_R2.02_full.fits' % data_dir)
planck_IQU_COM = hp.read_map('%sCOM_CMB_IQU-commander_1024_R2.02_full.fits' % data_dir)
planck_IQU_NILC = hp.read_map('%sCOM_CMB_IQU-nilc_1024_R2.02_full.fits' % data_dir)

In [ ]:
# best fit of data
(mu, sigma) = norm.fit(planck_IQU_SMICA)

# the histogram of the data
n, bins, patches = plt.hist(planck_IQU_SMICA, 60, normed=1, facecolor='r', alpha=0.25)

# add a 'best fit' line
y = mlab.normpdf( bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)

n, bins, patches = plt.hist(white_noise, 60, normed=1, facecolor='b', alpha=0.25)

plt.xlim(-0.0005, 0.0005)
plt.xlabel('Temperature/K')
plt.ylabel('Frequency')
plt.title(r'Histogram of $12N_{side}^2$ random samples from a normal (Gaussian) distribution and the Planck SMICA map', y=1.08)
plt.legend()
plt.show()

CMB Temperature fluctuations are tiny!

1 part in $10^5$



In [ ]:
fig = plt.figure(1)
hp.mollview(planck_IQU_SMICA, min=-0.0003, max=0.0003, title='Planck Temperature Map',
            fig=1, unit='K',cmap=cmap)

In [ ]:
# compute and plot the power spectrum

cl_SMICA = hp.anafast(planck_IQU_SMICA, lmax=1024)
cl_COM = hp.anafast(planck_IQU_COM, lmax=1024)
cl_NILC = hp.anafast(planck_IQU_NILC, lmax=1024)

ell = np.arange(len(cl_SMICA))


In [ ]:
pl = hp.sphtfunc.pixwin(1024)

In [ ]:
#Deconvolve the beam and the pixel window function
dl_SMICA = cl_SMICA/(B_l(10.0, ell)**2*pl[0:1025]**2)
dl_COM = cl_COM/(B_l(10.0, ell)**2*pl[0:1025]**2)
dl_NILC = cl_NILC/(B_l(10.0, ell)**2*pl[0:1025]**2)

#Apply scaling factors for plotting (convention)
dl_SMICA = (ell * (ell + 1) * dl_SMICA / (2*math.pi)) / 1e-12
dl_COM = (ell * (ell + 1) * dl_COM / (2*math.pi)) / 1e-12
dl_NILC = (ell * (ell + 1) * dl_NILC / (2*math.pi)) / 1e-12

In [ ]:
cl_white = hp.anafast(white_noise, lmax=1024)

In [ ]:
#Scaled as above
dl_white = (ell * (ell + 1) * cl_white / (2*math.pi)) / 1e-12

In [ ]:
fig = plt.figure(3)
ax = fig.add_subplot(111)
ax.scatter(ell, dl_SMICA,
           s=4, c='black', lw=0, 
           label='SMICA')
ax.scatter(ell, dl_COM,
           s=4, c='blue', lw=0, 
           label='COM')
ax.scatter(ell, dl_NILC,
           s=4, c='green', lw=0, 
           label='NILC')
ax.scatter(ell, dl_white,
           s=4, c='gray', lw=0,
           label='white noise')

ax.plot(planck_theory_cl[:,0], planck_theory_cl[:,1], '-r', label='Theory')


ax.set_xlabel('$\ell$')
ax.set_ylabel('$\ell(\ell+1)C_\ell/2\pi \,\,(\mu K^2)$ ')
ax.set_title('Angular Power Spectra')
ax.legend(loc='upper right')
ax.grid()

ax.set_xlim(2, 1024)
ax.set_ylim(2, 7000)

plt.show()

In [ ]:
fig = plt.figure(3)
ax = fig.add_subplot(111)
ax.scatter(ell, dl_SMICA,
           s=4, c='black', lw=0, 
           label='SMICA')
ax.scatter(ell, dl_COM,
           s=4, c='blue', lw=0, 
           label='COM')
ax.scatter(ell, dl_NILC,
           s=4, c='green', lw=0, 
           label='NILC')
ax.scatter(ell, dl_white,
           s=4, c='gray', lw=0,
           label='white noise')
ax.set_xscale('log')
ax.set_yscale('log')

ax.plot(planck_theory_cl[:,0], planck_theory_cl[:,1], '-r', label = 'Theory')


ax.set_xlabel('$\ell$')
ax.set_ylabel('$\ell(\ell+1)C_\ell/2\pi \,\,(\mu K^2)$ ')
ax.set_title('Angular Power Spectra')
ax.legend(loc='lower left')
ax.grid()

ax.set_xlim(2, 1024)
ax.set_ylim(2, 10000)

plt.show()

In [ ]:
hp.zoomtool.mollzoom(planck_IQU_SMICA, cmap=cmap)

In [ ]:
hp.zoomtool.mollzoom(white_noise, cmap=cmap)

CMB Simulations

Unfortunately, we can only observe one CMB. We can never observe them from another part of the Universe. 

So, in order to do experiments with the CMB, we need simulation... 

We will do this using healpy and the power spectrum from Planck

In [ ]:
nside =1024
lmax = 1024
mmax = 1024
 
synmap = hp.sphtfunc.synfast(cl_SMICA, nside, lmax=lmax, mmax=mmax, verbose=True)

fig = plt.figure(1)
hp.mollview(synmap,title='Simulated Universe', min = -0.0003, max = 0.0003,
            fig=1, unit=r'K', cmap=cmap)

Let's manually smooth the map with a Gaussian beam by convolving the power spectrum with the Gaussian beam (simple multiplication in Fourier space!)

In [ ]:
cl_obs = hp.anafast(synmap, lmax=1024)
ls = np.arange(len(cl_obs))
cl = cl_obs*B_l(30.0, ls)**2

alms = hp.sphtfunc.synalm(cl)
map_smoothed = hp.sphtfunc.alm2map(alms, 1024)

And plot it

In [ ]:
# plot the map
fig = plt.figure(1)
hp.mollview(map_smoothed,title='Manually smoothed map', min = -0.0003, max = 0.0003,
            fig=1, unit=r'K', cmap=cmap)

Using the healpy smoothing function gives similar results

In [ ]:
map_smoothed_2 = hp.sphtfunc.smoothing(synmap, fwhm = 0.5*math.pi/180)

In [ ]:
# plot the unmasked map
fig = plt.figure(1)
hp.mollview(map_smoothed_2, min = -0.0003, max = 0.0003,title='Smoothed map',
            fig=1, unit=r'K', cmap=cmap)

To zoom in use mollzoom

In [ ]:
hp.zoomtool.mollzoom(map_smoothed, cmap=cmap)

In [ ]:
hp.zoomtool.mollzoom(map_smoothed_2, cmap=cmap)

In [ ]:
hp.zoomtool.mollzoom(synmap, cmap=cmap)

The Planck website also provides maps of foregrounds e.g. interstellar dust:

In [ ]:
planck_dust = hp.read_map('%sCOM_CompMap_dust-commander_0256_R2.00.fits'  % data_dir)

In [ ]:
fig = plt.figure(1)
hp.mollview(planck_dust, min=0, max=1000, title='Thermal dust emission map',
            fig=1, cmap=cmap)